In [ ]:
import os
from getpass import getpass

import dotenv
from sqlalchemy import create_engine, engine

dotenv.load_dotenv()

# engine = create_engine('postgresql://localhost/example')
def connect_unix_socket() -> engine.base.Engine:
    """Initializes a Unix socket connection pool for a Cloud SQL instance of Postgres."""
    # Note: Saving credentials in environment variables is convenient, but not
    # secure - consider a more secure solution such as
    # Cloud Secret Manager (https://cloud.google.com/secret-manager) to help
    # keep secrets safe.
    db_user = os.environ["DB_USER"]  # e.g. 'my-database-user'
    db_pass = getpass('DB_PASS: ')  # e.g. 'my-database-password'
    db_name = os.environ["DB_NAME"]  # e.g. 'my-database'
    unix_socket_path = os.environ["INSTANCE_UNIX_SOCKET"]  # e.g. '/cloudsql/project:region:instance'

    pool = create_engine(
        # Equivalent URL:
        # postgresql+pg8000://<db_user>:<db_pass>@/<db_name>
        #                         ?unix_sock=<INSTANCE_UNIX_SOCKET>/.s.PGSQL.5432
        # Note: Some drivers require the `unix_sock` query parameter to use a different key.
        # For example, 'psycopg2' uses the path set to `host` in order to connect successfully.
        engine.url.URL.create(
            drivername="postgresql+psycopg2",
            username=db_user,
            password=db_pass,
            database=db_name,
            query={"host": unix_socket_path},
        ),
        # ...
    )
    return pool


In [ ]:
from sqlalchemy.orm import sessionmaker

engine = connect_unix_socket()
Session = sessionmaker(bind=engine)

In [ ]:
from sqlalchemy.orm import declarative_base

Base = declarative_base()

In [ ]:
from sqlalchemy import Column, Integer, String


class Records(Base):
    __tablename__ = "records"

    id = Column(Integer, primary_key=True, autoincrement=True, nullable=False)
    name = Column(String(255))
    # Add more columns as needed

In [ ]:
engine.url

In [ ]:
Base.metadata.create_all(engine)